In [13]:
import subprocess
import pysam

In [14]:
VCF2FASTA_FILEPATH = 'output/vcf2fasta.fasta'
ANALYSIS_SCRIPT = 'analysis.py'

'''
1. run analysis.py
2. get phase_change locations
3. run readcomb
4. sort readcomb output and analysis output
5. get read count per window
'''

# step 1

coverage = 1
percentage = 0.00004
analysis_output = 'output/testing'
base_count = 1000000

subprocess.run(['python3', 'analysis.py',
                '-f', VCF2FASTA_FILEPATH,
                '-e', str(base_count),
                '-c', str(coverage),
                '-p', str(percentage),
                '-o', analysis_output])

# step 2

phase_changes = str(subprocess.check_output(['tail', '-1', 'output/phase_change_log.txt']))

# step 3
filtered_output = 'output/filtered_testing'

subprocess.run(['readcomb-filter',
                '-b', analysis_output + '.sam',
                '-v', '../data/filtered_full.vcf.gz',
                '-p', '4',
                '-o', filtered_output])

# step 4

with open(analysis_output + 'sorted.bam', 'w') as f:
    subprocess.call(['samtools', 'sort', analysis_output + '.sam'], stdout=f)
    
subprocess.run(['samtools', 'index', analysis_output + 'sorted.bam'])

with open(filtered_output + 'sorted.bam', 'w') as f:
    subprocess.call(['samtools', 'sort', filtered_output + '.sam'], stdout=f)
    
subprocess.run(['samtools', 'index', filtered_output + 'sorted.bam'])


CompletedProcess(args=['samtools', 'index', 'output/filtered_testingsorted.bam'], returncode=0)

In [21]:
# step 5


unfiltered = pysam.AlignmentFile(analysis_output + 'sorted.bam', 'r')
filtered = pysam.AlignmentFile(filtered_output + 'sorted.bam', 'r')

counts = []

for i in range(0, base_count, 1000):
    counts.append([
        len(list(filtered.fetch('chromosome_1', i, i+2000))),
        len(list(unfiltered.fetch('chromosome_1', i, i+2000)))
    ])

with open('output/window_counts.csv', 'w') as f:
    for i in range(len(counts)):
        f.write(','.join(map(str, [
            i * 1000,
            (i + 2) * 1000,
            counts[i][0],
            counts[i][1]
        ])) + '\n')

In [ ]:
# create a 400 